In [1]:
using CSV, DataFrames, JuliaDB, Random, WiSER, vGWAS

┌ Info: Precompiling vGWAS [9514c204-b736-47c9-8157-11c3e9e5ab30]
└ @ Base loading.jl:1278


In [2]:
filepath = normpath(joinpath(dirname(pathof(WiSER)), "../data/"))
df = DataFrame!(CSV.File(filepath * "sbp.csv"))

,id,sbp,agegroup,gender,bmi,meds,bmi_std,obswt
,Int64,Float64,Float64,String,Float64,String,Float64,Float64
1,1,159.586,3.0,Male,23.1336,NoMeds,-1.57733,4.0
2,1,161.849,3.0,Male,26.5885,NoMeds,1.29927,4.0
3,1,160.484,3.0,Male,24.8428,NoMeds,-0.154204,4.0
4,1,161.134,3.0,Male,24.9289,NoMeds,-0.0825105,4.0
5,1,165.443,3.0,Male,24.8057,NoMeds,-0.185105,4.0
6,1,160.053,3.0,Male,24.1583,NoMeds,-0.72415,4.0
7,1,162.1,3.0,Male,25.2543,NoMeds,0.188379,4.0
8,1,163.153,3.0,Male,24.3951,NoMeds,-0.527037,4.0
9,1,166.675,3.0,Male,26.1514,NoMeds,0.935336,4.0


In [3]:
vlmm = WSVarLmmModel(
    @formula(sbp ~ 1 + agegroup + gender + bmi_std + meds), 
    @formula(sbp ~ 1 + bmi_std), 
    @formula(sbp ~ 1 + agegroup + meds + bmi_std),
    :id, df);

In [4]:
WiSER.fit!(vlmm)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

run = 1, ‖Δβ‖ = 0.037311, ‖Δτ‖ = 0.166678, ‖ΔL‖ = 0.100999, status = Optimal, time(s) = 0.386473
run = 2, ‖Δβ‖ = 0.005220, ‖Δτ‖ = 0.006748, ‖ΔL‖ = 0.048735, status = Optimal, time(s) = 0.237642



Within-subject variance estimation by robust regression (WiSER)
Number of individuals/clusters: 500
Total observations: 5011

Fixed-effects parameters:
───────────────────────────────────────────────────────────
                     Estimate  Std. Error       Z  Pr(>|Z|)
───────────────────────────────────────────────────────────
β1: (Intercept)   106.308       0.14384    739.07    <1e-99
β2: agegroup       14.9844      0.0633245  236.63    <1e-99
β3: gender: Male   10.0749      0.100279   100.47    <1e-99
β4: bmi_std         0.296424    0.0139071   21.31    <1e-99
β5: meds: OnMeds  -10.1107      0.122918   -82.26    <1e-99
τ1: (Intercept)    -2.5212      0.393792    -6.40    <1e-9
τ2: agegroup        1.50759     0.135456    11.13    <1e-28
τ3: meds: OnMeds   -0.435225    0.0621076   -7.01    <1e-11
τ4: bmi_std         0.0052695   0.0224039    0.24    0.8140
───────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  1.00196    

In [5]:
sum(x -> size(x.Xt, 2), vlmm.data)

5011

In [6]:
using Random
Random.seed!(333)
df[!, :rand1] = randn(size(df, 1));

In [7]:
X1vec = Matrix{Float64}[]
W1vec = Matrix{Float64}[]
for i in unique(df[!, :id])
    push!(X1vec, reshape(df[df[!, :id].==i, :rand1], :, 1))
    push!(W1vec, reshape(df[df[!, :id].==i, :rand1], :, 1))
end

In [8]:
st = vGWAS.WSVarScoreTest(vlmm, 1, 1)
vGWAS.test!(st, X1vec, W1vec)

(0.3833668584762737, 0.45613663515756114, 0.5030914655169818)

In [9]:
st = vGWAS.WSVarScoreTest(vlmm, 1, 0)
vGWAS.test!(st, X1vec, nothing)

(0.3833668584762737, -1.0, -1.0)

In [10]:
st = vGWAS.WSVarScoreTest(vlmm, 0, 1)
vGWAS.test!(st, nothing, W1vec)

(-1.0, 0.45613663515756114, -1.0)

In [ ]:
using Random
pvals1 = Float64[]
pvals2 = Float64[]
pvals3 = Float64[]
st = vGWAS.WSVarScoreTest(vlmm, 1, 1)
for i in 1:10000
    Random.seed!(37410+i)
    df[!, :rand1] = randn(size(df, 1))
    df[!, :rand2] = randn(size(df, 1))
    X1vec = Matrix{Float64}[]
    W1vec = Matrix{Float64}[]
    for i in unique(df[!, :id])
        push!(X1vec, reshape(df[df[!, :id].==i, :rand1], :, 1))
        push!(W1vec, reshape(df[df[!, :id].==i, :rand2], :, 1))
    end
    pval1, pval2, pval3 = vGWAS.test!(st, X1vec, W1vec)
    push!(pvals1, pval1)
    push!(pvals2, pval2)
    push!(pvals3, pval3)
end

In [ ]:
count(pvals1 .< 0.01), count(pvals2 .< 0.01), count(pvals3 .< 0.01)

In [ ]:
count(pvals1 .< 0.05), count(pvals2 .< 0.05), count(pvals3 .< 0.05)

In [ ]:
count(pvals1 .< 0.10), count(pvals2 .< 0.10), count(pvals3 .< 0.10)